Brave search: simple gradient tape model<br>


In [ ]:
import tensorflow as tf

# Define model parameters as trainable variables
a = tf.Variable(initial_value=0.0, trainable=True)
b = tf.Variable(initial_value=0.0, trainable=True)

# Define training data (X, Y)
X = tf.constant([1.0, 2.0, 3.0, 4.0])
Y = tf.constant([2.0, 4.0, 6.0, 8.0])

# Define the loss function
def loss_fn(y_true, y_pred):
    return tf.reduce_sum(tf.square(y_true - y_pred))

# Define the learning rate
learning_rate = 0.01

# Training loop
for epoch in range(100):
    # Record operations for automatic differentiation
    with tf.GradientTape() as tape:
        # Forward pass: compute predictions
        y_pred = a * X + b
        # Compute the loss
        loss = loss_fn(Y, y_pred)
    
    # Compute gradients of the loss with respect to the model parameters
    gradients = tape.gradient(loss, [a, b])
    
    # Update the parameters using the gradients
    a.assign_sub(gradients * learning_rate)
    b.assign_sub(gradients:inlineRefs{references="&#91;&#123;&quot;type&quot;&#58;&quot;inline_reference&quot;,&quot;start_index&quot;&#58;1545,&quot;end_index&quot;&#58;1548,&quot;number&quot;&#58;1,&quot;url&quot;&#58;&quot;https&#58;//www.linkedin.com/pulse/gradient-tape-deploy-descent-tensorflow-vu-hong-quan&quot;,&quot;favicon&quot;&#58;&quot;https&#58;//imgs.search.brave.com/B3dmoKTAgUkkbrvzAxFg_MfHAm5WmWy0N-4kH1AGWOo/rs&#58;fit&#58;32&#58;32&#58;1&#58;0/g&#58;ce/aHR0cDovL2Zhdmlj/b25zLnNlYXJjaC5i/cmF2ZS5jb20vaWNv/bnMvNGE1YzZjOWNj/NmNiODQ4NzI0ODg1/MGY5ZGQ2YzhjZTRm/N2NjOGIzZjc1NTlj/NDM2ZGI5Yjk3ZWI1/YzBmNzJmZS93d3cu/bGlua2VkaW4uY29t/Lw&quot;,&quot;snippet&quot;&#58;&quot;Gradient&#32;tape&#32;-&#32;deploy&#32;gradient&#32;descent&#32;with&#32;tensorflow\nTools&#32;for&#32;building&#32;machine&#32;learning&#32;and&#32;deep&#32;learning&#32;models&#32;such&#32;as&#32;Tensorflow&#32;and&#32;Pytorch&#32;are&#32;more&#32;and&#32;more&#32;popular.&#32;Building&#32;a&#32;model&#32;becomes&#32;easy&#32;with&#32;just&#32;one&#32;fit&#32;statement.\nTools&#32;for&#32;building&#32;machine&#32;learning&#32;and&#32;deep&#32;learning&#32;models&#32;such&#32;as&#32;Tensorflow&#32;and&#32;Pytorch&#32;are&#32;more&#32;and&#32;more&#32;popular.&#32;Building&#32;a&#32;model&#32;becomes&#32;easy&#32;with&#32;just&#32;one&#32;fit…&quot;&#125;&#93;"} * learning_rate)
    
    # Print loss every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy():.4f}")

# Output: The model parameters a and b will converge to values close to 2.0 and 0.0, respectively.   